In [7]:
import yfinance as yf

In [9]:
# Get the S&P 500 ticker data
sp500 = yf.Ticker("^GSPC")

In [10]:
# Query historical prices
sp500 = sp500.history(period = "max")

In [11]:
sp500

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1927-12-30 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
1928-01-03 00:00:00-05:00,17.760000,17.760000,17.760000,17.760000,0,0.0,0.0
1928-01-04 00:00:00-05:00,17.719999,17.719999,17.719999,17.719999,0,0.0,0.0
1928-01-05 00:00:00-05:00,17.549999,17.549999,17.549999,17.549999,0,0.0,0.0
1928-01-06 00:00:00-05:00,17.660000,17.660000,17.660000,17.660000,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-09-11 00:00:00-04:00,5496.419922,5560.410156,5406.959961,5554.129883,3839450000,0.0,0.0
2024-09-12 00:00:00-04:00,5557.479980,5600.709961,5535.500000,5595.759766,3655070000,0.0,0.0
2024-09-13 00:00:00-04:00,5603.339844,5636.270020,5601.649902,5626.020020,3500790000,0.0,0.0


In [12]:
sp500.index

DatetimeIndex(['1927-12-30 00:00:00-05:00', '1928-01-03 00:00:00-05:00',
               '1928-01-04 00:00:00-05:00', '1928-01-05 00:00:00-05:00',
               '1928-01-06 00:00:00-05:00', '1928-01-09 00:00:00-05:00',
               '1928-01-10 00:00:00-05:00', '1928-01-11 00:00:00-05:00',
               '1928-01-12 00:00:00-05:00', '1928-01-13 00:00:00-05:00',
               ...
               '2024-09-04 00:00:00-04:00', '2024-09-05 00:00:00-04:00',
               '2024-09-06 00:00:00-04:00', '2024-09-09 00:00:00-04:00',
               '2024-09-10 00:00:00-04:00', '2024-09-11 00:00:00-04:00',
               '2024-09-12 00:00:00-04:00', '2024-09-13 00:00:00-04:00',
               '2024-09-16 00:00:00-04:00', '2024-09-17 00:00:00-04:00'],
              dtype='datetime64[ns, America/New_York]', name='Date', length=24294, freq=None)